In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy import wcs

The Simple Imaging Polynomial (SIP) convention by Shupe et al. (2005) (http://fits.gsfc.nasa.gov/registry/sip/SIP_distortion_v1_0.pdf) implements distortion corrections as polynomials in FITS headers.

The polynomials are defined as: $${r_x \choose r_y} = \textrm{CD} {x+p_x \choose y+p_y}$$
Here, $\textrm{CD}$ is the $2\times2$ transformation matrix, made up of the $\textrm{CD1_1}$, $\textrm{CD1_2}$, $\textrm{CD2_1}$ and $\textrm{CD2_2}$ parameters. $r_x$ and $r_y$ are the intermediate world coordinates, computed from deprojecting right ascension $\alpha$ and declination $\delta$ around some reference point $\alpha_0$ and $\delta_0$, set by the $\textrm{CRVAL1}$ and $\textrm{CRVAL2}$ keywords. $x$ and $y$ are pixel coordinates, with origin $\textrm{CRPIX1}$ and $\textrm{CRPIX2}$, while $p_x(x, y)$ and $p_y(x, y)$ are the distortion polynomials.

The distortion polynomials are defined as:
$p_x(x, y) = \sum_{i, j} A_{i,j}x^i y^j$ and $p_y(x, y) = \sum_{i, j} B_{i,j}x^i y^j$, with $i+j\le \textrm{A_ORDER}$ and $i+j\le \textrm{B_ORDER}$.

A 3rd order polynomial would have the form: $p_x(x, y) = A_{00}+A_{10}x+A_{20}x^2+A_{30}x^3+A_{01}y+A_{11}xy+A_{21}x^2y+A_{02}y^2+A_{12}xy^2+A_{03}y^3$, as the values for $A_{31}$, $A_{32}$, $A_{33}$, $A_{22}$, $A_{23}$ and $A_{13}$ are all zero. Hence, the polynomial coefficients can be stored as an $(m+1)\times (m+1)$ matrix, where $m = \textrm{A_ORDER}$ and $m=\textrm{B_ORDER}$. Also note that $A_{00}$, $A_{01}$ and $A_{10}$ are zero, as these parameters are dealt with by the $\textrm{CD}$ matrix.

Given that the $\textrm{CD}$ matrix is involved, the full relation becomes:
$r_x = \textrm{CD1_1} (x+p_x(x, y)) + \textrm{CD1_2} (y+p_y(x, y))$, such that:
$$r_x = \textrm{CD1_1} x + \textrm{CD1_1}A_{02}x^2+\textrm{CD1_1}A_{11}xy+\textrm{CD1_1}A_{20}x^2$$

In [3]:
t = ascii.read("distort.dat")

In [4]:
ra0, dec0 = np.mean(t["ra"]), np.mean(t["dec"])
x0, y0 = 1548, 1040
dx, dy = t["x"]-x0, t["y"]-y0

w = wcs.WCS(naxis=2)
w.wcs.ctype = ["RA---ZEA", "DEC--ZEA"]
w.wcs.cd = [[-1.0, 0.0], [0.0, 1.0]]
w.wcs.crval = [ra0, dec0]
w.wcs.crpix = [0.0, 0.0]

p = w.wcs_world2pix(t["ra"], t["dec"], 1)
rx, ry = p[0], p[1]

In [26]:
def solve_linear_equation(a, b):
    q, r = np.linalg.qr(a)
    y = np.dot(q.T, b)
    x = np.linalg.solve(r, y)
    return x

In [66]:
n = len(t)
#a = np.array([[1.0, dx[i], dy[i], dx[i]**2, dx[i]*dy[i], dy[i]**2, dx[i]**3, dx[i]**2*dy[i], dx[i]*dy[i]**2, dy[i]**3] for i in range(n)])

In [112]:
a_order = 2
ixs, iys = np.meshgrid(np.arange(a_order+1), np.arange(a_order+1))
c=ixs+iys<=a_order
ix, iy = ixs[c], iys[c]
print(len(ix))

6


In [117]:
a = np.array([dx**ix[i]*dy**iy[i] for i in range(len(iy))]).T
print(a.shape)
print(ix, iy)

(1198, 6)
[0 1 2 0 1 0] [0 0 0 1 1 2]


In [114]:
ax = solve_linear_equation(a, rx)
ay = solve_linear_equation(a, ry)
print(ax, ay)

[-2.73742276e+00  5.38684996e-02  4.27509970e-07 -6.21988565e-03
  3.76065103e-08  1.31675057e-06] [-5.00823802e-01  6.25729025e-03  4.99514171e-07  5.38646056e-02
 -9.18231887e-07  4.76235885e-07]


In [126]:
jx = (ix==1) & (iy==0)
print(jx)

[False  True False False False False]


In [120]:
cd = np.array([[ax[1], ax[3]], [ay[1], ay[3]]])

In [125]:
np.linalg.inv(cd)

array([[18.31802262,  2.11522956],
       [-2.12794994, 18.31934689]])